<a href="https://colab.research.google.com/github/shatakshii25/Machine-Learning-Projects/blob/main/TransferLearningOfPretrainedModelsOnMNISTDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Libraries**

In [1]:
from tensorflow.keras.datasets import mnist

In [2]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from tensorflow.keras.utils import to_categorical

In [4]:
import numpy as np
import pandas as pd

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [7]:
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

**Reshaping the Data**

In [8]:
X_train = X_train.reshape((X_train.shape[0], 28, 28))
X_test = X_test.reshape((X_test.shape[0], 28, 28))

**Changing the type to float**

In [9]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

**Convert data to 3 channels**

In [10]:
X_train = np.stack((X_train,)*3, axis=-1)
X_test = np.stack((X_test,)*3, axis=-1)

In [11]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [12]:
X_train.shape, X_test.shape

((60000, 28, 28, 3), (10000, 28, 28, 3))

In [13]:
#data augmentation with generator
train_generator = ImageDataGenerator(
    #normalization of images
    rescale = 1./255,
    #augmention of images to avoid overfitting  
    rotation_range = 40, 
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest'
)

val_generator = ImageDataGenerator(rescale = 1./255)
train_iterator = train_generator.flow(X_train, y_train, batch_size=512, shuffle=True)
val_iterator = val_generator.flow(X_test, y_test, batch_size=512, shuffle=False)

**Importing modules for pretrained model**

In [14]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [16]:
from tensorflow.keras.models import Sequential

In [18]:
from tensorflow.keras.layers import Dense 

In [20]:
model = Sequential()
# adding the pretrained model
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))

94781440/94765736 [==============================] - 0s 0us/step


In [22]:
# adding fully connected layer with output
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [23]:
# setting resnet layers not trainable
model.layers[0].trainable=False

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_2 (Dense)             (None, 512)               262656    
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                                 
Total params: 24,904,586
Trainable params: 1,316,874
Non-trainable params: 23,587,712
_________________________________________________________________


In [25]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

**Training the model**

In [26]:
model.fit(train_iterator, epochs = 10, validation_data=val_iterator)

Epoch 1/10
118/118 [==============================] - 149s 1s/step - loss: 1.3534 - accuracy: 0.5582 - val_loss: 0.7476 - val_accuracy: 0.7568
Epoch 2/10
118/118 [==============================] - 146s 1s/step - loss: 0.7797 - accuracy: 0.7483 - val_loss: 0.5772 - val_accuracy: 0.8110
Epoch 3/10
118/118 [==============================] - 146s 1s/step - loss: 0.6275 - accuracy: 0.7974 - val_loss: 0.4325 - val_accuracy: 0.8621
Epoch 4/10
118/118 [==============================] - 147s 1s/step - loss: 0.5493 - accuracy: 0.8233 - val_loss: 0.4173 - val_accuracy: 0.8581
Epoch 5/10
118/118 [==============================] - 147s 1s/step - loss: 0.5240 - accuracy: 0.8295 - val_loss: 0.4289 - val_accuracy: 0.8572
Epoch 6/10
118/118 [==============================] - 147s 1s/step - loss: 0.4563 - accuracy: 0.8521 - val_loss: 0.3689 - val_accuracy: 0.8743
Epoch 7/10
118/118 [==============================] - 147s 1s/step - loss: 0.4279 - accuracy: 0.8612 - val_loss: 0.3872 - val_accuracy: 0.8686